In [126]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from src.Utils import *
%matplotlib inline 

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
import os 

In [127]:
df_sample_sb = pd.read_csv('Data/sample_submission.csv')
df_stores = pd.read_csv('Data/stores.csv')
df_trans = pd.read_csv('Data/transactions.csv')
df_holi = pd.read_csv('Data/holidays_events.csv')
df_oil = pd.read_csv('Data/oil.csv')

df_train = pd.read_csv('Data/train.csv',nrows=200000)
df_test = pd.read_csv('Data/test.csv')


In [128]:
print(df_sample_sb.isnull().sum())
print(df_stores.isnull().sum())

id            0
unit_sales    0
dtype: int64
store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64


In [129]:
print(df_sample_sb.shape, df_stores.shape, df_trans.shape,
      df_oil.shape, df_train.shape, df_test.shape)

(3370464, 2) (54, 5) (83488, 3) (1218, 2) (200000, 6) (3370464, 5)


In [130]:
dataframes = [df_trans, df_holi, df_oil, df_train, df_test]
for df in dataframes:
    to_dateTime(df)

In [131]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           200000 non-null  int64         
 1   date         200000 non-null  datetime64[ns]
 2   store_nbr    200000 non-null  int64         
 3   item_nbr     200000 non-null  int64         
 4   unit_sales   200000 non-null  float64       
 5   onpromotion  0 non-null       float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 9.2 MB


In [132]:
merged_data = pd.merge(df_train, df_trans, on=['date', 'store_nbr'])
merged_data2 = pd.merge(merged_data, df_holi, on='date')
merged_data3 = pd.merge(merged_data2, df_oil, on='date')
merged_data4 = pd.merge(merged_data3, df_stores, on='store_nbr')
merged_data4.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,type_x,locale,locale_name,description,transferred,dcoilwtico,city,state,type_y,cluster
0,0,2013-01-01,25,103665,7.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1


In [133]:
new_merged_data = merged_data4.rename(
    columns={"type_x": "holiday_type", "type_y": "store_type"})
new_merged_data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,locale,locale_name,description,transferred,dcoilwtico,city,state,store_type,cluster
0,0,2013-01-01,25,103665,7.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1


In [134]:
print(new_merged_data.isnull().sum())
print(new_merged_data.info())

id                0
date              0
store_nbr         0
item_nbr          0
unit_sales        0
onpromotion     578
transactions      0
holiday_type      0
locale            0
locale_name       0
description       0
transferred       0
dcoilwtico      578
city              0
state             0
store_type        0
cluster           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            578 non-null    int64         
 1   date          578 non-null    datetime64[ns]
 2   store_nbr     578 non-null    int64         
 3   item_nbr      578 non-null    int64         
 4   unit_sales    578 non-null    float64       
 5   onpromotion   0 non-null      float64       
 6   transactions  578 non-null    int64         
 7   holiday_type  578 non-null    object        
 8   locale        578 non-null    object 

In [135]:
new_merged_data.to_csv("Data/new_merged_data.csv",index=False)

In [136]:
new_merged_data.date = pd.to_datetime(new_merged_data.date)


new_merged_data['year'] = new_merged_data.date.dt.year
new_merged_data['month'] = new_merged_data.date.dt.month

new_merged_data['dayofmonth'] = new_merged_data.date.dt.day

new_merged_data['dayofweek'] = new_merged_data.date.dt.dayofweek

new_merged_data['dayname'] = new_merged_data.date.dt.strftime('%A')

In [137]:
df_train = df_train.drop(['onpromotion'],axis=1)
df_train.isnull().sum()

id            0
date          0
store_nbr     0
item_nbr      0
unit_sales    0
dtype: int64

In [138]:
if df_train.isnull().values.any():
  print("The dataset is not complete. There are missing values.")

if not df_train.index.is_unique:
  print("The dataset is not complete. There are duplicate dates.")
else:
  print("The dataset is complete.")

The dataset is complete.


In [139]:
grouped_by_year = new_merged_data.groupby("year")["unit_sales"].agg(["min", "max"])

result = pd.concat([new_merged_data[new_merged_data["unit_sales"] == grouped_by_year.loc[year, "min"]][["year", "date"]].rename(columns={"date": "date_min"}) for year in grouped_by_year.index] +
                   [new_merged_data[new_merged_data["unit_sales"] == grouped_by_year.loc[year, "max"]][["year", "date"]].rename(columns={"date": "date_max"}) for year in grouped_by_year.index])

result = result.set_index("year")

result = result.groupby(level=0).agg(
    {"date_min": "first", "date_max": "first"})

result = result.reset_index()

print(result)

   year   date_min   date_max
0  2013 2013-01-01 2013-01-01


In [140]:
new_merged_data.duplicated().sum()

0

In [141]:
new_merged_data = new_merged_data.drop(
    columns=["year", "month", "dayofmonth", "dayofweek", "dayname"])
new_merged_data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,locale,locale_name,description,transferred,dcoilwtico,city,state,store_type,cluster
0,0,2013-01-01,25,103665,7.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,NaN,770,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Salinas,Santa Elena,D,1


In [142]:
new_merged_data.date = pd.to_datetime(new_merged_data.date)


new_merged_data['year'] = new_merged_data.date.dt.year

new_merged_data['month'] = new_merged_data.date.dt.month


new_merged_data['dayofmonth'] = new_merged_data.date.dt.day


new_merged_data['dayofweek'] = new_merged_data.date.dt.dayofweek


new_merged_data['dayname'] = new_merged_data.date.dt.strftime('%A')

In [143]:
new_merged_data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,locale,locale_name,...,dcoilwtico,city,state,store_type,cluster,year,month,dayofmonth,dayofweek,dayname
0,0,2013-01-01,25,103665,7.0,NaN,770,Holiday,National,Ecuador,...,NaN,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday
1,1,2013-01-01,25,105574,1.0,NaN,770,Holiday,National,Ecuador,...,NaN,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday
2,2,2013-01-01,25,105575,2.0,NaN,770,Holiday,National,Ecuador,...,NaN,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday
3,3,2013-01-01,25,108079,1.0,NaN,770,Holiday,National,Ecuador,...,NaN,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday
4,4,2013-01-01,25,108701,1.0,NaN,770,Holiday,National,Ecuador,...,NaN,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday


In [144]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

imputer.fit(new_merged_data[['dcoilwtico']])

new_merged_data['dcoilwtico'] = imputer.transform(
    new_merged_data[['dcoilwtico']])

c:\Users\anisr\anaconda3\envs\work\lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['dcoilwtico']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [145]:
final_data = new_merged_data.drop(
    columns=['id', 'locale', 'locale_name', 'description', 'transferred'], inplace=True)

In [146]:
df_items = pd.read_csv('Data/items.csv')
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [147]:
new_merged_data = pd.merge(new_merged_data, df_items, on='item_nbr')
new_merged_data.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,dcoilwtico,city,state,store_type,cluster,year,month,dayofmonth,dayofweek,dayname,family,class,perishable
0,2013-01-01,25,103665,7.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,BREAD/BAKERY,2712,1
1,2013-01-01,25,105574,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1045,0
2,2013-01-01,25,105575,2.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1045,0
3,2013-01-01,25,108079,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1030,0
4,2013-01-01,25,108701,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,DELI,2644,1


In [148]:
new_merged_data.set_index('date', inplace=True)
new_merged_data.head()

,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,dcoilwtico,city,state,store_type,cluster,year,month,dayofmonth,dayofweek,dayname,family,class,perishable
date,,,,,,,,,,,,,,,,,,,
2013-01-01,25,103665,7.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,BREAD/BAKERY,2712,1
2013-01-01,25,105574,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1045,0
2013-01-01,25,105575,2.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1045,0
2013-01-01,25,108079,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,GROCERY I,1030,0
2013-01-01,25,108701,1.0,NaN,770,Holiday,8.239533e-320,Salinas,Santa Elena,D,1,2013,1,1,1,Tuesday,DELI,2644,1


In [149]:
final_data = new_merged_data.drop(
    columns=['state',  'store_type', 'dayname'], inplace=True)
final_data = new_merged_data.copy()
final_data.head()

,store_nbr,item_nbr,unit_sales,onpromotion,transactions,holiday_type,dcoilwtico,city,cluster,year,month,dayofmonth,dayofweek,family,class,perishable
date,,,,,,,,,,,,,,,,
2013-01-01,25,103665,7.0,NaN,770,Holiday,8.239533e-320,Salinas,1,2013,1,1,1,BREAD/BAKERY,2712,1
2013-01-01,25,105574,1.0,NaN,770,Holiday,8.239533e-320,Salinas,1,2013,1,1,1,GROCERY I,1045,0
2013-01-01,25,105575,2.0,NaN,770,Holiday,8.239533e-320,Salinas,1,2013,1,1,1,GROCERY I,1045,0
2013-01-01,25,108079,1.0,NaN,770,Holiday,8.239533e-320,Salinas,1,2013,1,1,1,GROCERY I,1030,0
2013-01-01,25,108701,1.0,NaN,770,Holiday,8.239533e-320,Salinas,1,2013,1,1,1,DELI,2644,1


In [150]:
final_data.isnull().sum()

store_nbr         0
item_nbr          0
unit_sales        0
onpromotion     578
transactions      0
holiday_type      0
dcoilwtico        0
city              0
cluster           0
year              0
month             0
dayofmonth        0
dayofweek         0
family            0
class             0
perishable        0
dtype: int64

In [155]:
imputer = SimpleImputer(strategy='mean')

imputer.fit(final_data[['onpromotion']])

final_data['onpromotion'] = imputer.transform(final_data[['onpromotion']])

In [157]:
final_data.isnull().sum()

store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
transactions    0
holiday_type    0
dcoilwtico      0
city            0
cluster         0
year            0
month           0
dayofmonth      0
dayofweek       0
family          0
class           0
perishable      0
dtype: int64

In [158]:
food_families = ['BEVERAGES', 'BREAD/BAKERY', 'FROZEN FOODS', 'MEATS',
                 'PREPARED FOODS', 'DELI', 'PRODUCE', 'DAIRY', 'POULTRY', 'EGGS', 'SEAFOOD']
final_data['family'] = np.where(final_data['family'].isin(
    food_families), 'FOODS', final_data['family'])
home_families = ['HOME AND KITCHEN I',
                 'HOME AND KITCHEN II', 'HOME APPLIANCES']
final_data['family'] = np.where(final_data['family'].isin(
    home_families), 'HOME', final_data['family'])
clothing_families = ['LINGERIE', 'LADYSWARE']
final_data['family'] = np.where(final_data['family'].isin(
    clothing_families), 'CLOTHING', final_data['family'])
grocery_families = ['GROCERY I', 'GROCERY II']
final_data['family'] = np.where(final_data['family'].isin(
    grocery_families), 'GROCERY', final_data['family'])
stationery_families = ['BOOKS', 'MAGAZINES', 'SCHOOL AND OFFICE SUPPLIES']
final_data['family'] = np.where(final_data['family'].isin(
    stationery_families), 'STATIONERY', final_data['family'])
cleaning_families = ['HOME CARE', 'BABY CARE', 'PERSONAL CARE']
final_data['family'] = np.where(final_data['family'].isin(
    cleaning_families), 'CLEANING', final_data['family'])
hardware_families = ['PLAYERS AND ELECTRONICS', 'HARDWARE']
final_data['family'] = np.where(final_data['family'].isin(
    hardware_families), 'HARDWARE', final_data['family'])

In [162]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

num_cols = ['unit_sales', 'transactions', 'dcoilwtico',
            'year', 'month', 'dayofmonth', 'dayofweek']

final_data[num_cols] = scaler.fit_transform(final_data[num_cols])

In [163]:
from sklearn.preprocessing import OneHotEncoder

categorical_columns = ["family", "city", "holiday_type"]
categorical_data = final_data[categorical_columns]

encoder = OneHotEncoder()

one_hot_encoded_data = encoder.fit_transform(categorical_data)

categories = [encoder.categories_[i] for i in range(len(encoder.categories_))]

column_names = []
for i in range(len(categories)):
    for j in range(len(categories[i])):
        column_names.append(f'{categorical_columns[i]}_{categories[i][j]}')

one_hot_encoded_data = pd.DataFrame(
    one_hot_encoded_data.toarray(), columns=column_names)


final_data = final_data.reset_index(drop=True)
one_hot_encoded_data = one_hot_encoded_data.reset_index(drop=True)

final_data_encoded = pd.concat([final_data, one_hot_encoded_data], axis=1)

final_data_encoded.drop(categorical_columns, axis=1, inplace=True)

In [164]:
final_data_encoded.head()

,store_nbr,item_nbr,unit_sales,onpromotion,transactions,dcoilwtico,cluster,year,month,dayofmonth,...,perishable,family_BEAUTY,family_CLEANING,family_CLOTHING,family_FOODS,family_GROCERY,family_LAWN AND GARDEN,"family_LIQUOR,WINE,BEER",city_Salinas,holiday_type_Holiday
0,25,103665,0.397839,3.725293e-08,0.0,0.0,1,0.0,0.0,0.0,...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,25,105574,-0.501354,3.725293e-08,0.0,0.0,1,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,25,105575,-0.351489,3.725293e-08,0.0,0.0,1,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,25,108079,-0.501354,3.725293e-08,0.0,0.0,1,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,25,108701,-0.501354,3.725293e-08,0.0,0.0,1,0.0,0.0,0.0,...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
